In [ ]:
!wget https://github.com/opalchonlapat/anime-recommend/raw/master/anime.zip # Download dataset
!unzip anime.zip -d anime-recommedations-datbase

from fastai.collab import *
from fastai.tabular.all import *
from fastai import *
from ipywidgets import interact, interactive, fixed

import numpy as np
import pandas as pd
import torch
import ipywidgets as widgets

rating_df = pd.read_csv("anime-recommedations-datbase/rating.csv")
anime_df = pd.read_csv("anime-recommedations-datbase/anime.csv")

rating_df = rating_df[rating_df.rating >= 0].reset_index(drop=True) # filter unrating anime

# filter anime rated user >= 15
user_grouped = rating_df.groupby(['anime_id'])['user_id'].count().reset_index(name='num_user')
anime_id_min = user_grouped.loc[user_grouped['num_user'] >= 15, 'anime_id'].values
rating_df_min = rating_df[rating_df['anime_id'].isin(anime_id_min)]

# filter user rated anime >= 10
user_grouped = rating_df_min.groupby(['user_id'])['anime_id'].count().reset_index(name='num_rate_anime')
user_id_min = user_grouped.loc[user_grouped['num_rate_anime'] >= 10, 'user_id'].values
rating_df_min = rating_df_min[rating_df_min['user_id'].isin(user_id_min)] 

rating_grouped_df = rating_df_min.groupby(['user_id'])
valid_df = rating_grouped_df.apply(lambda x: x.sample(frac=.2, random_state=1))
valid_df.index = valid_df.index.droplevel(level=0)
train_df = rating_df_min[~rating_df_min.index.isin(valid_df.index)]

valid_df = valid_df.reset_index(drop=True)
valid_grouped_df = valid_df.groupby(['user_id'])
test_df = valid_grouped_df.apply(lambda x: x.sample(frac=.5, random_state=1))
test_df.index = test_df.index.droplevel(level=0)
valid_df = valid_df[~valid_df.index.isin(test_df.index)]

n_users, n_animes, n_ratings = list(train_df.nunique())

train_df = train_df.copy()
train_df['is_valid'] = 0

valid_df = valid_df.copy()
valid_df['is_valid'] = 1

train_valid_df = pd.concat([train_df, valid_df], axis=0)

user_name   = 'user_id'
item_name   = 'anime_id'
rating_name = 'rating'
cat_names = [user_name,item_name]
splits = ColSplitter(col='is_valid')(train_valid_df)
to = TabularCollab(train_valid_df, [Categorify], cat_names, y_names=[rating_name], y_block=TransformBlock(), splits=splits)
dls = to.dataloaders(bs=256)

emb_szs = {'user_id': 50, 'anime_id': 50}
szs = get_emb_sz(dls.train_ds, emb_szs)
learn = collab_learner(dls, # Dataloader ที่จะนำมา train model
                       y_range=(0.5, 10.5), # Range ของค่าคะแนน rating
                       use_nn=True, # ใช้ Neural Network
                       emb_szs=emb_szs, # กำหนดขนาดของ Embedding
                       metrics=[mse, # Metrics ที่ใช้วัดผล
                                mae]).to_fp16()

# load model
learn.load('model')

# Get embedding
user_embeds = learn.model.embeds[0].weight
item_embeds = learn.model.embeds[1].weight
user_ids = learn.dls.train_ds.classes['user_id']
item_ids = learn.dls.train_ds.classes['anime_id']

def find_similar_anime(anime_name):
    print(f"Anime name: {anime_name}")
    ani_id = anime_df.loc[anime_df['name'] == anime_name].iat[0, 0]
    arr_idx = item_ids.o2i[ani_id]
    similar = nn.CosineSimilarity(dim=1)(item_embeds, item_embeds[arr_idx][None])
    sim_arr_idx = similar.argsort(descending=True)
    most_sim_idx = sim_arr_idx[1:6]
    least_sim_idx = sim_arr_idx[-5:]
    sim_ani_id = np.concatenate((item_ids[most_sim_idx], item_ids[least_sim_idx]))

    sim_df = pd.DataFrame()
    for id in sim_ani_id:
        sim_df = sim_df.append(anime_df[anime_df['anime_id'] == id])
    sim_df['similar'] = np.concatenate((similar[most_sim_idx].cpu().detach().numpy(), similar[least_sim_idx].cpu().detach().numpy()))
    return sim_df

anime_cnt = rating_df.groupby(['anime_id'])['user_id'].count().reset_index(name='cnt')
anime_id_cnt = anime_cnt.sort_values(['cnt'], ascending=False).loc[:50, 'anime_id'].values

names_arr = anime_df.loc[anime_df['anime_id'].isin(anime_id_cnt), 'name'].unique()
interact(find_similar_anime, anime_name=names_arr)